  Author: Ankit Kariryaa, University of Bremen
  
  Modified by Xuehui Pi and Qiuqi Luo

### Getting started
Define the paths for the dataset and trained models in the `notebooks/config/UNetTraining.py` file.  

In [1]:
import os
os.environ["MKL_NUM_THREADS"] = '16'
os.environ["NUMEXPR_NUM_THREADS"] = '16'
os.environ["OMP_NUM_THREADS"] = '16'
print(os.environ.get('OMP_NUM_THREADS'))

16


In [2]:
import tensorflow as tf
import numpy as np
from PIL import Image
import rasterio
import imgaug as ia
from imgaug import augmenters as iaa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import mixed_precision 
mixed_precision.set_global_policy('mixed_float16')

import imageio
import os

import time
import rasterio.warp             # Reproject raster samples
from functools import reduce
from tensorflow.keras.models import load_model

from core.UNet import UNet  #
from core.losses import tversky, focalTversky, bce_dice_loss, accuracy, dice_loss, IoU, recall, precision,F1_score
from tensorflow.keras.losses import BinaryCrossentropy as bce
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.frame_info import FrameInfo
from core.dataset_generator import DataGenerator
from core.visualize import display_images_2,plot
import json
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pickle
import random

%matplotlib inline
import matplotlib.pyplot as plt  # plotting tools
import matplotlib.patches as patches
from matplotlib.patches import Polygon
#matplotlib.use("Agg")

import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Mixed precision is the use of both 16-bit and 32-bit floating-point types in a model during training to make it run faster and use less memory.
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
print(tf.__version__)

2024-01-21 16:11:27.846054: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA RTX A6000, compute capability 8.6


2024-01-21 16:11:29.146365: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-01-21 16:11:29.147307: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-01-21 16:11:29.176034: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-01-21 16:11:29.176058: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-01-21 16:11:29.178481: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-01-21 16:11:29.178546: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2024-0

2.4.1


In [3]:
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# tf.device('/gpu:1')

In [4]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2024-01-21 16:11:30.544047: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-21 16:11:30.545904: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-01-21 16:11:30.546281: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-01-21 16:11:30.546321: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-01-21 16:11:30.546412: I tensorflow/stream_executor/platform/default/dso_loader.cc

In [5]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/UNetTraining.py)
# Please provide required info in the file before continuing with this notebook. 
# hbh: in this scene,a new config named UNetTraining_sequential is created to distinguish from the original
from config import UNetTraining_b5
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import UNetTraining
config = UNetTraining_b5.Configuration()

In [6]:
# def readBands(path_to_write,fn):
#     img=rasterio.open(os.path.join(path_to_write,fn))
# #     im=img.read()
# #     axis=(0, 1)
# #     read_img=(im - im.mean(axis)) / (im.std(axis) + 1e-8)
#     read_img=img.read()/1000
#     return read_img

# def readImgs(path_to_write, fn):
#     NDWI_img = rasterio.open(os.path.join(path_to_write, fn))
#     read_NDWI_img = NDWI_img.read()/100
#     rowNum=read_NDWI_img.shape[1]/config.patch_size[0]
#     colNum=read_NDWI_img.shape[2]/config.patch_size[1]
#     read_red_img =readBands(path_to_write,fn.replace(config.NDWI_fn ,config.red_fn))
#     read_green_img =readBands(path_to_write,fn.replace(config.NDWI_fn ,config.green_fn))
#     read_blue_img = readBands(path_to_write,fn.replace(config.NDWI_fn ,config.blue_fn))
#     read_swir_img = readBands(path_to_write, fn.replace(config.NDWI_fn ,config.swir_fn))
#     comb_img = np.concatenate((read_NDWI_img,read_red_img,read_green_img,read_blue_img, read_swir_img), axis=0)
#     comb_img = np.transpose(comb_img, axes=(1,2,0)) #Channel at the end  ( , ,1) 
    
#     annotation_im = Image.open(os.path.join(path_to_write, fn.replace(config.NDWI_fn,config.annotation_fn)))
#     annotation = np.array(annotation_im)
    
#     f = FrameInfo(comb_img, annotation)
#     return f,rowNum*colNum

# def readFrames(dataType):
#     frames=[]
#     numList=[]
#     print(dataType)
# #     for i in range(1,4):
#     for i in range(0,config.type_num):
#         path_to_write=os.path.join(config.dataset_dir,'{}/type{}'.format(dataType,i))
#         all_files = os.listdir(path_to_write)
#         all_files_NDWI = [fn for fn in all_files if fn.startswith(config.NDWI_fn) and fn.endswith(config.image_type)]#ndwi.png
#         print('type{} image number:{}'.format(i,len(all_files_NDWI)))
#         for j, fn in enumerate(all_files_NDWI):
#             f,num = readImgs(path_to_write,fn)
#             if i==0:
#                 num=num
#             frames.append(f)
#             numList.append(num)
#     return frames,numList

In [7]:
# def readNum(dataType):       
#     numList=[]
#     for i in range(0,config.type_num):
#         path_to_write=os.path.join(config.dataset_dir,'{}/type{}'.format(dataType,i))
#         all_files = os.listdir(path_to_write)
#         all_files_NDWI = [fn for fn in all_files if fn.startswith(config.NDWI_fn) and fn.endswith(config.image_type)]#ndwi.png
#         print('type{} image number:{}'.format(i,len(all_files_NDWI)))
#         for j, fn in enumerate(all_files_NDWI):
#             NDWI_img = rasterio.open(os.path.join(path_to_write, fn))
#             read_NDWI_img = NDWI_img.read()
#             rowNum=read_NDWI_img.shape[1]/config.patch_size[0]
#             colNum=read_NDWI_img.shape[2]/config.patch_size[1]
#             numList.append(rowNum*colNum)
    
    # annotation_im = Image.open(os.path.join(path_to_write, fn.replace(config.NDWI_fn,config.annotation_fn)))
#     return numList

In [8]:
def readImgs(path_to_write, fn):
    image = rasterio.open(os.path.join(path_to_write, fn))
#     print(fn)
    read_image = image.read()/1000
#     print(read_image[1].max())
    
    comb_img = np.transpose(read_image, axes=(1,2,0))
    annotation_im = Image.open(os.path.join(path_to_write, fn.replace(config.image_fn,config.annotation_fn).replace(config.image_type,config.ann_type)))
    annotation = np.array(annotation_im)
    rowNum=annotation.shape[0]/config.input_shape[0]
    colNum=annotation.shape[1]/config.input_shape[1]
    f = FrameInfo(comb_img, annotation)
    return f,rowNum*colNum
    
def readFrames(dataType):
    frames=[]
    numList=[]
    print(dataType)
#     for i in range(3,4):
    for i in range(0,config.type_num):#config.type_num
        path_to_write=os.path.join(config.dataset_dir,'{}/type{}'.format(dataType,i))
        all_files = os.listdir(path_to_write)
        all_files_image = [fn for fn in all_files if fn.startswith(config.image_fn) and fn.endswith(config.image_type)]#image.png
        print('type{} image number:{}'.format(i,len(all_files_image)))
        for j, fn in enumerate(all_files_image):
            f,num = readImgs(path_to_write,fn)
            frames.append(f)
            numList.append(num)
    return frames,numList


## 参数初始化

In [9]:
OPTIMIZER = adaDelta
OPTIMIZER =  mixed_precision.LossScaleOptimizer(OPTIMIZER)
OPTIMIZER_NAME = 'AdaDelta'

# OPTIMIZER = adam
# OPTIMIZER = mixed_precision.LossScaleOptimizer(OPTIMIZER)
# OPTIMIZER_NAME = 'adam'

2024-01-21 16:11:45.768920: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-01-21 16:11:45.769394: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-01-21 16:11:45.769432: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-01-21 16:11:45.769502: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-01-21 16:11:45.769513: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2024-01-21 16:11:45.769523: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2024

In [10]:
# LOSS = tversky 
# LOSS_NAME = 'tversky'

# LOSS=focalTversky
# LOSS_NAME = 'focalTversky'

#LOSS=tf.keras.losses.BinaryCrossentropy()
#LOSS_NAME = 'bce'

# LOSS=bce_dice_loss
# LOSS_NAME = 'bce_dice_loss'

LOSS=dice_loss
LOSS_NAME = 'dice_loss'


## 数据集准备

In [11]:
frames,numList=readFrames('train')
percentages=np.array(numList)
train_sum=percentages.sum()
percentages=percentages/percentages.sum()
print('total training img count:'+str(len(frames)))
print('total training patches count:{}'.format(train_sum))
train_generator = DataGenerator(config.input_image_channel, config.patch_size, frames, config.input_label_channel, augmenter = 'iaa').random_generator(config.BATCH_SIZE,percentages)#,normalize =

train
type0 image number:165
type1 image number:142
type2 image number:68
type3 image number:53
type4 image number:17
total training img count:445
total training patches count:7907.519546320408


In [12]:
# numList=readNum('train')
# percentages=np.array(numList)
# print(percentages.sum())

In [12]:
frames,numList=readFrames('val')
percentages=np.array(numList)
val_sum=percentages.sum()
percentages=percentages/percentages.sum()
print('total training img count:'+str(len(frames)))
print('total training patches count:{}'.format(val_sum))
val_generator = DataGenerator(config.input_image_channel, config.patch_size, frames, config.input_label_channel, augmenter = None).random_generator(config.BATCH_SIZE,percentages)#, normalize = config.normalize

val
type0 image number:56
type1 image number:47
type2 image number:24
type3 image number:18
type4 image number:4
total training img count:149
total training patches count:2563.5930627893517


In [14]:
# print(numList)

In [ ]:
for _ in range(1):
    train_images, real_label = next(train_generator) 
#     print(train_images.Length())
    display_images_2(np.concatenate((train_images,real_label), axis = -1),pad=100,output_size=476)

In [ ]:
for _ in range(1):
    val_images, val_label = next(val_generator) 
    print(val_images.shape)
    display_images_2(np.concatenate((val_images,val_label), axis = -1),pad=84,output_size=600)

### 训练参数设置

In [14]:
timestr = time.strftime("%Y%m%d-%H%M")
chf = config.input_image_channel + config.input_label_channel
chs = reduce(lambda a,b: a+str(b), chf, '') 

if not os.path.exists(config.model_path):
    os.makedirs(config.model_path)
model_name='_{}_{}_{}_{}_{}.h5'.format(timestr,OPTIMIZER_NAME,LOSS_NAME,chs,config.input_shape[0])
model_path = os.path.join(config.model_path,'lakes'+model_name)

chf = config.input_image_channel + config.input_label_channel
chs = reduce(lambda a,b: a+str(b), chf, '') 
print(model_path)

/home/nkd/hbh/5_saved_models/lakes_20240121-1616_AdaDelta_dice_loss_012345_576.h5


In [15]:
# Define the model and compile it  
model = UNet([config.BATCH_SIZE, *config.input_shape],config.input_label_channel)
# model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_loss, accuracy, recall, precision, IoU])
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[accuracy, recall, precision,F1_score, IoU])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 576, 576, 5) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 576, 576, 64) 2944        Input[0][0]                      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 576, 576, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 576, 576, 64) 256         conv2d_1[0][0]                   
______________________________________________________________________________________________

In [16]:
# Define callbacks      for the early stopping of training, LearningRateScheduler and model checkpointing 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = False)

#reduceonplatea： It can be useful when using adam as optimizer
#Reduce learning rate when a metric has stopped improving (after some patience 个epoch, reduce by a factor of 0.33, new_lr = lr * factor). 
#cooldown: number of epochs to wait before resuming normal operation after lr has been reduced. 

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.33,
                                   patience=4, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=4, min_lr=1e-16) 

early = EarlyStopping(monitor="val_loss", mode="min", verbose=2, patience=40)


log_dir = os.path.join('./logs','UNet'+model_name)
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

callbacks_list = [checkpoint, tensorboard] #reduceLROnPlat is not required with adaDelta, early

2024-01-21 16:16:00.430599: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2024-01-21 16:16:00.430624: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2024-01-21 16:16:00.430650: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1365] Profiler found 1 GPUs
2024-01-21 16:16:00.430951: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcupti.so.11.0'; dlerror: libcupti.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nkd/anaconda3/envs/hbh_env/lib/python3.8/site-packages/cv2/../../lib64::/usr/local/cuda/lib64
2024-01-21 16:16:00.431039: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nkd/anaconda3/envs/hbh_env/lib/python3.8/site-packages/cv2/../../lib64::/us

## 训练模型
保存history文件并绘图

In [17]:
loss_history = model.fit(train_generator, 
                         steps_per_epoch=train_sum//config.BATCH_SIZE,#config.MAX_TRAIN_STEPS,
                         epochs=config.NB_EPOCHS, 
                         validation_data=val_generator,
                         validation_steps=val_sum//config.BATCH_SIZE,#config.VALID_IMG_COUNT,
                         callbacks=callbacks_list,
                         workers=1,
                         # shuffle=True,
#                          use_multiprocessing=True # the generator is not very thread safe 
                         #max_queue_size = 60,
                        )
h=loss_history.history
with open('history_{}_{}_{}_{}_{}.txt'.format(timestr,OPTIMIZER_NAME,LOSS_NAME, chs,config.input_shape[0]), 'wb') as file_pi:
    pickle.dump(h, file_pi)
plot(h,timestr, OPTIMIZER_NAME,LOSS_NAME, config.patch_size[0], config.NB_EPOCHS, config.BATCH_SIZE,chs)

2024-01-21 16:16:00.896159: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-01-21 16:16:00.915055: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000000000 Hz


Epoch 1/150


2024-01-21 16:16:02.977237: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2024-01-21 16:16:05.217868: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-21 16:16:05.218091: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: /usr/local/cuda-11.0/bin/ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-01-21 16:16:05.610529: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The pt

  1/494 [..............................] - ETA: 33:40:14 - loss: 0.9825 - accuracy: 0.0159 - recall: 0.9966 - precision: 0.0083 - F1_score: 0.0165 - IoU: 0.0083

2024-01-21 16:20:06.782224: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-21 16:20:06.866841: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2024-01-21 16:20:06.866871: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2024-01-21 16:20:06.866931: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.


  2/494 [..............................] - ETA: 7:16 - loss: 0.9662 - accuracy: 0.0262 - recall: 0.9974 - precision: 0.0147 - F1_score: 0.0287 - IoU: 0.0147    

2024-01-21 16:20:07.685457: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2024-01-21 16:20:07.685594: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1496] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2024-01-21 16:20:07.699861: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:228]  GpuTracer has collected 0 callback api events and 0 activity events. 
2024-01-21 16:20:07.705005: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2024-01-21 16:20:07.711990: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/UNet_20240121-1616_AdaDelta_dice_loss_012345_576.h5/train/plugins/profile/2024_01_21_16_20_07
2024-01-21 16:20:07.715355: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to ./logs/UNet_20240121-1616_AdaDelta_dice_loss_012345_576.h5/train/p

494/494 [==============================] - 736s 995ms/step - loss: 0.3536 - accuracy: 0.9507 - recall: 0.7993 - precision: 0.6065 - F1_score: 0.6514 - IoU: 0.5204 - val_loss: 0.2807 - val_accuracy: 0.9848 - val_recall: 0.6972 - val_precision: 0.7676 - val_F1_score: 0.7196 - val_IoU: 0.5765

Epoch 00001: val_loss improved from inf to 0.28066, saving model to /home/nkd/hbh/5_saved_models/lakes_20240121-1616_AdaDelta_dice_loss_012345_576.h5
Epoch 2/150
494/494 [==============================] - 499s 1s/step - loss: 0.2402 - accuracy: 0.9845 - recall: 0.8745 - precision: 0.6946 - F1_score: 0.7602 - IoU: 0.6315 - val_loss: 0.2887 - val_accuracy: 0.9789 - val_recall: 0.9116 - val_precision: 0.6077 - val_F1_score: 0.7115 - val_IoU: 0.5723

Epoch 00002: val_loss did not improve from 0.28066
Epoch 3/150
494/494 [==============================] - 507s 1s/step - loss: 0.2044 - accuracy: 0.9869 - recall: 0.8858 - precision: 0.7428 - F1_score: 0.7959 - IoU: 0.6762 - val_loss: 0.2012 - val_accuracy:

625/625 [==============================] - ETA: 0s - loss: 0.0397 - accuracy: 0.9978 - recall: 0.9590 - precision: 0.9624 - F1_score: 0.9604 - IoU: 0.9245
Epoch 127: val_loss did not improve from 0.07498
625/625 [==============================] - 541s 865ms/step - loss: 0.0397 - accuracy: 0.9978 - recall: 0.9590 - precision: 0.9624 - F1_score: 0.9604 - IoU: 0.9245 - val_loss: 0.0910 - val_accuracy: 0.9952 - val_recall: 0.8970 - val_precision: 0.9282 - val_F1_score: 0.9091 - val_IoU: 0.8413
Epoch 128/150
625/625 [==============================] - ETA: 0s - loss: 0.0414 - accuracy: 0.9978 - recall: 0.9584 - precision: 0.9596 - F1_score: 0.9587 - IoU: 0.9214
Epoch 128: val_loss did not improve from 0.07498
625/625 [==============================] - 540s 863ms/step - loss: 0.0414 - accuracy: 0.9978 - recall: 0.9584 - precision: 0.9596 - F1_score: 0.9587 - IoU: 0.9214 - val_loss: 0.1127 - val_accuracy: 0.9932 - val_recall: 0.8941 - val_precision: 0.8954 - val_F1_score: 0.8873 - val_IoU: 0.8

625/625 [==============================] - ETA: 0s - loss: 0.0374 - accuracy: 0.9980 - recall: 0.9619 - precision: 0.9639 - F1_score: 0.9626 - IoU: 0.9286
Epoch 143: val_loss did not improve from 0.07498
625/625 [==============================] - 543s 868ms/step - loss: 0.0374 - accuracy: 0.9980 - recall: 0.9619 - precision: 0.9639 - F1_score: 0.9626 - IoU: 0.9286 - val_loss: 0.0912 - val_accuracy: 0.9949 - val_recall: 0.9150 - val_precision: 0.9135 - val_F1_score: 0.9089 - val_IoU: 0.8431
Epoch 144/150
625/625 [==============================] - ETA: 0s - loss: 0.0382 - accuracy: 0.9980 - recall: 0.9610 - precision: 0.9634 - F1_score: 0.9619 - IoU: 0.9273
Epoch 144: val_loss did not improve from 0.07498
625/625 [==============================] - 543s 869ms/step - loss: 0.0382 - accuracy: 0.9980 - recall: 0.9610 - precision: 0.9634 - F1_score: 0.9619 - IoU: 0.9273 - val_loss: 0.0870 - val_accuracy: 0.9948 - val_recall: 0.9133 - val_precision: 0.9214 - val_F1_score: 0.9131 - val_IoU: 0.8

In [17]:
##### 读取现有history文件
with open(r'G:\5_lakemapping\U_Net\notebooks\history\6type_747\history_20240109-1249_AdaDelta_dice_loss_012345_576.txt','rb')as file_pi:
    h=pickle.load(file_pi)
# print(h)

In [15]:
timestr='20240109-1249'
chs='012345'
plot(h,timestr, OPTIMIZER_NAME,LOSS_NAME, config.patch_size[0], config.NB_EPOCHS, config.BATCH_SIZE,chs)

## 模型精度评价

In [23]:
# Load model after training 
model_path=os.path.join(config.model_path,r'lakes_20240118-1337_AdaDelta_dice_loss_012345_576.h5')
model = load_model(model_path, custom_objects={'dice loss': LOSS, 'accuracy':accuracy ,'recall':recall, 'precision':precision,'F1_score':F1_score,'IoU': IoU,}, compile=False) 
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_loss, accuracy,recall,F1_score, precision, IoU])

In [12]:
# # Print one batch on the training/test data! 
# for i in range(1):
#     test_images, real_label = next(test_generator)
#     #3 images per row: GSW, label, prediction
#     prediction = model.predict(test_images, steps=1)
#     prediction[prediction>0.5]=1
#     prediction[prediction<=0.5]=0
#     display_images(np.concatenate((test_images, real_label, prediction), axis = -1))# test_images( NDWI), real_label(label), prediction

In [ ]:
titles=['ndwi','rgb','swir','annotation','prediction']
j=1
frames=[]
path_to_write=os.path.join(config.dataset_dir,'test/type'+str(j))
all_files = os.listdir(path_to_write)
all_image_files = [fn for fn in all_files if fn.startswith(config.image_fn) and fn.endswith(config.image_type)]#ndwi.png
for i, fn in enumerate(all_image_files):
    f,nums = readImgs(path_to_write,fn)
    frames.append(f)
test_DGT=DataGenerator(config.input_image_channel, config.patch_size, frames, config.input_label_channel, augmenter = None)
test_patches_type = test_DGT.all_sequential_patches(config.step_size)
print(len(test_patches_type[0]))
step=8
for i in range(len(test_patches_type[0])//step+1):
    prediction = model.predict(test_patches_type[0][i*step:(i+1)*step], steps=1)
    prediction[prediction>0.5]=1
    prediction[prediction<=0.5]=0
    fn=r'D:\sample746\image\type{}_{}_576_2.png'.format(j,i)
    image=np.concatenate((test_patches_type[0][i*step:(i+1)*step], test_patches_type[1][i*step:(i+1)*step], prediction), axis = -1)
    display_images_2(image,92,388,fn,titles=titles)

## 总体精度评价

In [18]:
frames,numList=readFrames('test')
test_patches = DataGenerator(config.input_image_channel, config.patch_size, frames, config.input_label_channel, augmenter = None).all_sequential_patches(config.step_size)
print('test patchs number:',len(test_patches[0]))

test
type0 image number:53
type1 image number:48
type2 image number:23
type3 image number:18
type4 image number:3
test patchs number: 2864


In [24]:
ev=model.evaluate(test_patches[0],test_patches[1],config.BATCH_SIZE)
ev.append(len(test_patches[0]))

179/179 [==============================] - 40s 220ms/step - loss: 0.0691 - dice_loss: 0.0691 - accuracy: 0.9968 - recall: 0.9147 - F1_score: 0.9309 - precision: 0.9559 - IoU: 0.8770


In [20]:
#0121 改了数据集
print(ev)

[0.07784055918455124, 0.9966059923171997, 0.9219013452529907, 0.9319897294044495, 0.9222298860549927, 0.8635604381561279, 2864]


In [25]:
#0118 没改
print(ev)

[0.08337188512086868, 0.08332246541976929, 0.9963952302932739, 0.9199349284172058, 0.9166976809501648, 0.9288750290870667, 0.8584253191947937, 2864]


In [20]:
test_patchs=[]

## 分类别精度评价

In [26]:
ev_list=[]
for i in range(0,config.type_num):
    frames=[]
    test_patchs_type=[]
    path_to_write=os.path.join(config.dataset_dir,'test/type'+str(i))
    all_files = os.listdir(path_to_write)
    all_files_NDWI = [fn for fn in all_files if fn.startswith(config.image_fn) and fn.endswith(config.image_type)]#ndwi.png
    for j, fn in enumerate(all_files_NDWI):
        f ,num = readImgs(path_to_write,fn)
        frames.append(f)
    test_patchs_type = DataGenerator(config.input_image_channel, config.patch_size, frames, config.input_label_channel, augmenter = None).all_sequential_patches(config.step_size)
    print('type{} patches number:{}'.format(i,len(test_patchs_type[0])))
    ev2 = model.evaluate(test_patchs_type[0],test_patchs_type[1],config.BATCH_SIZE,verbose=1)
    ev2.append(len(test_patchs_type[0]))
    ev_list.append(ev2)
    del frames,test_patchs_type

type0 patches number:1099
69/69 [==============================] - 16s 225ms/step - loss: 0.0535 - dice_loss: 0.0532 - accuracy: 0.9977 - recall: 0.9261 - F1_score: 0.9468 - precision: 0.9725 - IoU: 0.9015
type1 patches number:912
57/57 [==============================] - 13s 223ms/step - loss: 0.1012 - dice_loss: 0.1011 - accuracy: 0.9943 - recall: 0.9071 - F1_score: 0.8989 - precision: 0.9029 - IoU: 0.8294
type2 patches number:453
29/29 [==============================] - 7s 220ms/step - loss: 0.1122 - dice_loss: 0.1110 - accuracy: 0.9981 - recall: 0.9215 - F1_score: 0.8890 - precision: 0.8806 - IoU: 0.8163
type3 patches number:383
24/24 [==============================] - 6s 224ms/step - loss: 0.0976 - dice_loss: 0.0976 - accuracy: 0.9962 - recall: 0.9399 - F1_score: 0.9025 - precision: 0.9019 - IoU: 0.8480
type4 patches number:17
2/2 [==============================] - 0s 24ms/step - loss: 0.1236 - dice_loss: 0.1376 - accuracy: 0.9863 - recall: 0.9009 - F1_score: 0.8616 - precision: 0.

In [28]:
print(ev_list)

[[0.053475912660360336, 0.05323919281363487, 0.9976851940155029, 0.926115095615387, 0.9467891454696655, 0.9724763631820679, 0.9014774560928345, 1099], [0.10116908699274063, 0.10111967474222183, 0.9942695498466492, 0.9070848822593689, 0.8988975286483765, 0.902921736240387, 0.8294309377670288, 912], [0.1121668815612793, 0.11103816330432892, 0.9980668425559998, 0.9215235114097595, 0.888960063457489, 0.8805680274963379, 0.816290020942688, 453], [0.09762141108512878, 0.09755461663007736, 0.9962234497070312, 0.939880907535553, 0.9024580121040344, 0.9018687605857849, 0.8480146527290344, 383], [0.12361351400613785, 0.13761168718338013, 0.9863366484642029, 0.9009413719177246, 0.8615968823432922, 0.8264418244361877, 0.7572259902954102, 17]]


In [22]:
#0121改了数据集
print('      patch_num   loss accuracy   recall  precision F1_score     IoU')
print('total:{:^9}{:^9.3f}{:^9.3f}{:^9.3f} {:^9.3f} {:^9.3f} {:^9.3f}'.format(ev[6],ev[0],ev[1],ev[2],ev[3],ev[4],ev[5]))
for i in range(0,config.type_num):
    print('type{}:{:^9}{:^9.3f}{:^9.3f}{:^9.3f} {:^9.3f} {:^9.3f} {:^9.3f}'.format(i,ev_list[i][6],ev_list[i][0],ev_list[i][1],ev_list[i][2],ev_list[i][3],ev_list[i][4],ev_list[i][5])

      patch_num   loss accuracy   recall  precision F1_score     IoU
total:  2864     0.078    0.997    0.922     0.932     0.922     0.864  
type0:  1099     0.057    0.997    0.924     0.969     0.943     0.896  
type1:   912     0.101    0.995    0.910     0.897     0.899     0.828  
type2:   453     0.099    0.998    0.924     0.894     0.902     0.830  
type3:   383     0.063    0.997    0.948     0.930     0.937     0.883  
type4:   17      0.120    0.987    0.858     0.846     0.852     0.743  


In [31]:
#0118 没改
print('      patch_num   loss accuracy   recall  precision F1_score     IoU')
print('total:{:^9}{:^9.3f}{:^9.3f}{:^9.3f} {:^9.3f} {:^9.3f} {:^9.3f}'.format(ev[7],ev[0],ev[2],ev[3],ev[4],ev[5],ev[6]))
for i in range(0,config.type_num):
    print('type{}:{:^9}{:^9.3f}{:^9.3f}{:^9.3f} {:^9.3f} {:^9.3f} {:^9.3f}'.format(i,ev_list[i][7],ev_list[i][0],ev_list[i][2],ev_list[i][3],ev_list[i][4],ev_list[i][5],ev_list[i][6]))

      patch_num   loss accuracy   recall  precision F1_score     IoU
total:  2864     0.083    0.996    0.920     0.917     0.929     0.858  
type0:  1099     0.053    0.998    0.926     0.947     0.972     0.901  
type1:   912     0.101    0.994    0.907     0.899     0.903     0.829  
type2:   453     0.112    0.998    0.922     0.889     0.881     0.816  
type3:   383     0.098    0.996    0.940     0.902     0.902     0.848  
type4:   17      0.124    0.986    0.901     0.862     0.826     0.757  


In [ ]:
del frames,test_patchs,test_patch_type

In [ ]:
%tensorboard --logdir logs